In [6]:
from improved_find_data_start_row import *

from financials_clean_year_assign import * 

financial_data=improved_load_excel_with_autodetect("/Users/myself/Desktop/Walmart USA Serching for Growth/walmartHistoricalFinancials.xlsx")

In [7]:
# Correctly identify and assign unique years to each financial data column
# Assuming the first column after 'Financial Metric' is the most recent year (2021) and decrement for each column after

number_of_years = financial_data.shape[1] - 1  # Total columns minus the 'Financial Metric' column
base_year = 2021
years = [str(base_year - i) for i in range(number_of_years)]

# Map the new year labels to the columns
financial_data.columns = ['Financial Metric'] + years

In [8]:
import numpy as np

financial_data['Financial Metric']=financial_data['Financial Metric'].str.replace('∟', '', regex=True)
financial_data.replace('\xa0', '', regex=True, inplace=True)
financial_data.replace('n.a.', np.nan, inplace=True)
financial_data['Financial Metric']=financial_data['Financial Metric'].str.strip()
financial_data=financial_data.dropna(thresh=3)

In [9]:
financial_data=clean_cashflow_data(financial_data)
financial_data=update_column_names(financial_data, base_year=2021,index_name = 'Financial Metric')

In [10]:
# df = financial_data.copy()

# if df.index.name != 'Financial Metric':
#     dfcopy=df
#     df.set_index('Financial Metric', inplace=True)  # Setting 'Financial Metric' as index


# # Determine categories based on keyword presence or row structure
# assets = [metric for metric in df.index if "asset" in metric.lower()]
# liabilities_and_equity = [metric for metric in df.index if "liabilit" in metric.lower() or "equity" in metric.lower() or "fund" in metric.lower()]
# p_and_l = [metric for metric in df.index if metric not in assets and metric not in liabilities_and_equity]

# # Example to show categorized lists
# print("Assets:", assets)
# print("Liabilities & Equity:", liabilities_and_equity)
# print("P&L Items:", p_and_l)

In [11]:
# # Create a base mapping dictionary
# base_mapping = {}

# # Map assets to 'Total assets'
# for item in assets:
#     base_mapping[item] = 'Total assets'

# # Map liabilities and equity to 'Total shareholders' funds and liabilities'
# for item in liabilities_and_equity:
#     base_mapping[item] = 'Total shareholders\' funds and liabilities'

# # Map P&L items to 'Sales' (this could be adjusted to 'Operating revenue' if more appropriate)
# for item in p_and_l:
#     if 'Revenue' in item or 'Sales' in item:
#         base_mapping[item] = 'Sales'
#     else:
#         base_mapping[item] = 'Sales'  # or 'Total operating revenue' if defined in your dataset

# # Print or return the base mapping
# base_mapping.items()



In [12]:
from regularModelFunctions import base_mapping

categoried, base_mapping = base_mapping(financial_data)

In [13]:
df.index.name
df.columns

NameError: name 'df' is not defined

## Forecast Base Financial Metrics with Assumed YOY Rates

In [14]:
# Define the growth assumptions based on the details provided earlier
assumptions = {
    'Sales': {'type': 'YOY', 'rates': [0.0814, 0.0392, 0.0261, 0.0244, 0.0237, 0.0238]},
    'Costs of goods sold': {'type': '% of revenue', 'rates': [0.7541, 0.7513, 0.7513, 0.7513, 0.7513, 0.7513]},
    'Gross profit': {'type': 'direct', 'rates': None},  # Calculated directly as difference
    'Operating and SG&A costs': {'type': '% of sales', 'rates': [0.1863, 0.1834, 0.1834, 0.1834, 0.1834, 0.1834]}
}

# Base year for forecast
base_year = '2021'
forecast_years = ['2022F','2023F', '2024F', '2025F', '2026F', '2027F']

# Initialize a DataFrame to hold the forecasted values
yoy_forecast_df = pd.DataFrame(index=df.index, columns=forecast_years)

# Iterate over each forecast year and apply the assumptions to calculate forecast values
for idx, year in enumerate(forecast_years):
    if 'Sales' in assumptions:
        if idx == 0:  # First forecast year, base it on the last historical year
            last_sales = df.loc[df.index.str.contains("Sales"), base_year].values[0]
        growth_rate = assumptions['Sales']['rates'][idx]
        forecast_sales = last_sales * (1 + growth_rate)
        yoy_forecast_df.loc['Sales', year] = forecast_sales
        last_sales = forecast_sales  # Update for next year's calculation

    if 'Costs of goods sold' in assumptions:
        cost_rate = assumptions['Costs of goods sold']['rates'][idx]
        forecast_costs = forecast_sales * cost_rate
        yoy_forecast_df.loc['Costs of goods sold', year] = forecast_costs

    if 'Gross profit' in assumptions:
        forecast_gross_profit = forecast_sales - forecast_costs
        yoy_forecast_df.loc['Gross profit', year] = forecast_gross_profit

    if 'Operating and SG&A costs' in assumptions:
        sgna_rate = assumptions['Operating and SG&A costs']['rates'][idx]
        forecast_sgna = forecast_sales * sgna_rate
        yoy_forecast_df.loc['Operating and SG&A costs', year] = forecast_sgna

yoy_forecast_df = yoy_forecast_df / 1e3
# Calculate CAGR for the forecast period

yoy_cagr_df = (yoy_forecast_df[forecast_years].astype(float).iloc[:, -1] / yoy_forecast_df[forecast_years].astype(float).iloc[:, 0]) ** (1 / (len(forecast_years) - 1)) - 1
yoy_forecast_df.loc[:, 'CAGR'] = yoy_cagr_df*100


# Display the forecast results
yoy_forecast_df






NameError: name 'df' is not defined

In [15]:
# # # Define growth assumptions based on the details provided earlier
# assumptions = {
#     'Sales': {'type': 'YOY', 'rates': [0.0814, 0.0392, 0.0261, 0.0244, 0.0237, 0.0238]},
#     'Costs of goods sold': {'type': '% of revenue', 'rates': [0.7541, 0.7513, 0.7513, 0.7513, 0.7513, 0.7513]},
#     'Gross profit': {'type': 'direct', 'rates': None},  # Calculated directly as difference
#     'Operating and SG&A costs': {'type': '% of sales', 'rates': [0.1863, 0.1834, 0.1834, 0.1834, 0.1834, 0.1834]}
# }

# # # # Calculate future bases from assumptions
# # base_year = '2021'
# # forecast_years = ['2022F', '2023F', '2024F', '2025F', '2026F', '2027F']
# # last_values = df[base_year]

# # for year in forecast_years:
# #     for metric, config in assumptions.items():
# #         if config['type'] == 'YOY':
# #             growth_rate = config['rates'][int(year[:4]) - 2022]
# #             forecast_value = last_values[metric] * (1 + growth_rate)
# #         elif config['type'] == '% of revenue':
# #             revenue_based_rate = config['rates'][int(year[:4]) - 2022]
# #             forecast_value = forecast_df.loc['Sales', year] * revenue_based_rate
# #         elif config['type'] == 'direct':
# #             forecast_value = forecast_df.loc['Sales', year] - forecast_df.loc['Costs of goods sold', year]
# #         forecast_df.loc[metric, year] = forecast_value
# #         last_values[metric] = forecast_value


## Handle NaNs

In [16]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler


if 'Costs of employees' not in df.columns: df = df.T
# Assuming df is your DataFrame loaded with data

# Identify columns with missing values
cols_with_missing = df.columns[df.isnull().any()].tolist()

# Decide on an imputation strategy for each column
# Example: Use mean for numerical data and a model for 'Costs of employees'
imputation_strategies = {
    col: ('model' if df[col].dtype.kind in 'biufc' else 'most_frequent')
    for col in cols_with_missing if col != 'Costs of employees'
}
imputation_strategies['Costs of employees'] = 'model'  # Deciding to use a model for 'Costs of employees'

# Apply imputation or model prediction
for col, strategy in imputation_strategies.items():
    if strategy != 'model':
        # Simple imputation
        imputer = SimpleImputer(strategy=strategy)
        df[col] = imputer.fit_transform(df[[col]])
    else:
        # Setup for predictive modeling
        # Assuming you've already identified features to use
        features = df.columns.difference(cols_with_missing).tolist()
        train_data = df.dropna(subset=[col] + features)
        target = train_data[col]
        train_features = train_data[features]

        # Scaling features
        scaler = StandardScaler()
        train_features_scaled = scaler.fit_transform(train_features)
        
        # Model fitting
        model = RandomForestRegressor(random_state=0)
        model.fit(train_features_scaled, target)
        
        # Predicting missing values
        test_features = df.loc[df[col].isnull(), features]
        test_features_scaled = scaler.transform(test_features)
        predicted_values = model.predict(test_features_scaled)
        
        # Fill in the missing values
        df.loc[df[col].isnull(), col] = predicted_values

print("Missing values handled for columns:", cols_with_missing)
print(df.info())


NameError: name 'df' is not defined

In [ ]:
df.columns

## Forecast all values on Historical Data

In [ ]:
if '2021' not in df.columns: df = df.T
from sklearn.linear_model import LinearRegression

# Forecast years and base year definition
forecast_years = ['2022F', '2023F', '2024F', '2025F', '2026F', '2027F']
historical_years = df.columns[df.columns.str.isnumeric()]

# Forecast function
def forecast_metric(values, years, forecast_years):
    years_reshaped = years.values.reshape(-1, 1)
    values_reshaped = values.values.reshape(-1, 1)
    model = LinearRegression()
    model.fit(years_reshaped, values_reshaped)
    future_years = pd.Series(forecast_years).str[:-1].astype(int).values.reshape(-1, 1)
    predictions = model.predict(future_years).flatten()
    return pd.Series(predictions, index=forecast_years)

# Apply forecasting
historical_regression_forecast = df.apply(lambda x: forecast_metric(x, historical_years, forecast_years), axis=1) / 1e3
historical_cagr_values = (historical_regression_forecast[forecast_years].astype(float).iloc[:, -1] / historical_regression_forecast[forecast_years].astype(float).iloc[:, 0]) ** (1 / (len(forecast_years) - 1)) - 1
historical_regression_forecast.loc[:, 'CAGR'] = historical_cagr_values*100


# Print results
historical_regression_forecast

## Forecast Using Ratios of Non Base Financial Metrics to Base Metrics

In [ ]:
# from sklearn.linear_model import LinearRegression
# import numpy as np
# #Assuming 'financial_data' is your DataFrame
# if df.index.name != 'Financial Metric':
#     dfcopy=df
#     df.set_index('Financial Metric', inplace=True)  # Setting 'Financial Metric' as index

# # Calculate historical ratios
# historical_ratios = {}
# for metric, base in base_mapping.items():
#     print(metric,base)
#     if metric in df.index and base in df.index:
#         historical_ratios[metric] = df.loc[metric] / df.loc[base]
# # Example of performing linear regression on one of the metrics

# print(historical_ratios)
# model = LinearRegression()
# if 'Operating profit (loss) [EBIT]' in historical_ratios:
#     ratios = historical_ratios['Operating profit (loss) [EBIT]'].dropna()  # Drop NaN to avoid fitting errors
#     years = np.array([int(year) for year in ratios.index]).reshape(-1, 1)
#     model.fit(years, ratios.values.reshape(-1, 1))
#     # Predict for a future year, e.g., 2022
#     future_year = np.array([[2022]])
#     forecast_ratio = model.predict(future_year)
#     print(f"Forecasted Ratio for 2022: {forecast_ratio[0][0]}")

# # Use similar logic for other metrics


In [5]:

# Now apply these dynamic bases to your regression forecasts
future_bases = {
    'Sales': historical_regression_forecast.loc['Sales'],
    'Total assets': historical_regression_forecast.loc['Total assets']  # Assuming 'Total assets' forecast is calculated similarly
}
print(future_bases)
#Calculate historical ratios for regression
historical_ratios = {}
for metric, base in base_mapping.items():
    historical_ratios[metric] = df.loc[metric] / df.loc[base]

historical_ratios=pd.DataFrame.from_dict(historical_ratios)
historical_ratios=historical_ratios.dropna(axis='columns')

#for metric, ratios in historical_ratios.items():
#    print(ratios.index.tolist(), ratios.values.tolist())

# Perform regression and forecast future ratios
forecasts = {}
projected_ratios = {}

yeardf = pd.DataFrame(df.columns)


for metric, ratios in historical_ratios.items():
    model = LinearRegression()
    X = yeardf.values.reshape(-1, 1)  # Ensure X is correctly shaped
    y = ratios

    # Fit the linear model
    model.fit(X, y)
    historical_signs = np.sign(ratios).tolist()
    last_valid_value = ratios[0]# Start with the last historical value

    # Create X and y for model fitting
    for year in forecast_years:
        projected_ratio = model.predict(np.array([[int(year[:-1])]]))[0]
         # Forecast each future year
        projected_sign = np.sign(projected_ratio)
        # Check if the projected sign is not in historical signs
        if projected_sign not in historical_signs:
            #print(metric, projected_sign, historical_signs, int(year[:-1]))
            # If mismatch, use the last valid value
            projected_ratio = last_valid_value
            #print(f"Sign mismatch for {metric} in {year}; using last valid value {last_valid_value}")
        else:
            # If match, update last valid value
            last_valid_value = projected_ratio

        # Store the forecasts
        forecasts[metric, year] = projected_ratio * future_bases[base][year]
        projected_ratios[metric, year] = projected_ratio


NameError: name 'historical_regression_forecast' is not defined

In [ ]:
# Process the dictionary to fit DataFrame constructor
# Create an empty list to hold the data
processed_data = []

# Iterate through the dictionary to populate the list
for (metric, year), value in projected_ratios.items():
    processed_data.append({
        'Financial Metric': metric,
        'Year': year,
        'Value': value  # Convert numpy array to a scalar
    })

forecasts_data = []
for (metric, year), value in forecasts.items():
    forecasts_data.append({
        'Financial Metric': metric,
        'Year': year,
        'Value': value  # Convert numpy array to a scalar
    })





# Create a DataFrame from the list


ratio_regression_forecast = pd.DataFrame(forecasts_data)
ratio_regression_percentages = pd.DataFrame(processed_data)


# Pivot the DataFrame to get years as columns


result_ratio_regression_forecast = ratio_regression_forecast.pivot(index='Financial Metric', columns='Year', values='Value')
cagr_ratio_regression_forecast = (result_ratio_regression_forecast[forecast_years].astype(float).iloc[:, -1] / result_ratio_regression_forecast[forecast_years].astype(float).iloc[:, 0]) ** (1 / (len(forecast_years) - 1)) - 1
result_ratio_regression_forecast.loc[:, 'CAGR'] = cagr_ratio_regression_forecast * 100

result_ratio_regression_percentages = ratio_regression_percentages.pivot(index='Financial Metric', columns='Year', values='Value')*100



# Display the resulting DataFrame

result_ratio_regression_forecast.loc['Sales'], result_ratio_regression_percentages.loc['Costs of goods sold']

In [ ]:
# output_file_path = '/Users/myself/Desktop/Walmart USA Serching for Growth/BaselineForecasts.xlsx'




# yoy_cagr_df = (yoy_forecast_df[forecast_years].astype(float).iloc[:, -1] / yoy_forecast_df[forecast_years].astype(float).iloc[:, 0]) ** (1 / (len(forecast_years) - 1)) - 1
# yoy_forecast_df=yoy_forecast_df.astype(float).round(0)
# yoy_forecast_df.loc[:, 'CAGR'] = yoy_cagr_df.round(4)*100

# historical_cagr_values = (historical_regression_forecast[forecast_years].astype(float).iloc[:, -1] / historical_regression_forecast[forecast_years].astype(float).iloc[:, 0]) ** (1 / (len(forecast_years) - 1)) - 1
# historical_regression_forecast=historical_regression_forecast.astype(float).round(0)
# historical_regression_forecast.loc[:, 'CAGR'] = historical_cagr_values.round(4)*100

# cagr_ratio_regression_forecast = (result_ratio_regression_forecast[forecast_years].astype(float).iloc[:, -1] / result_ratio_regression_forecast[forecast_years].astype(float).iloc[:, 0]) ** (1 / (len(forecast_years) - 1)) - 1
# result_ratio_regression_forecast=result_ratio_regression_forecast.astype(float).round(0)
# result_ratio_regression_forecast.loc[:, 'CAGR'] = cagr_ratio_regression_forecast.round(4) * 100

# result_ratio_regression_percentages=result_ratio_regression_percentages.astype(float).round(4)

# with pd.ExcelWriter(output_file_path) as writer:
#     yoy_forecast_df.to_excel(writer, sheet_name='CAGR Raw')
#     historical_regression_forecast.to_excel(writer, sheet_name='Historical Regression')
#     result_ratio_regression_forecast.to_excel(writer, sheet_name='Historical Base Val Reg')
#     result_ratio_regression_percentages.to_excel(writer, sheet_name='Historical Base Rat Reg')

# print("File printed to: " + output_file_path)

# yoy_forecast_df